> need to run ch1 to load data.

In [1]:
import os
import sqlite3
import numpy as np
import pandas as pd
import sklearn

In [2]:
import nltk
import re

In [3]:
#### ICT Contracts in the last 3 years
contract = pd.read_csv('3ycontract.csv') # main contract tabel
(contract.head())

,Cnid,idSupplier,Title,PDate,austender_cat,austender_supplier,internalcomment,EntityName,IdCat
0,CN1148392,9446,Provision of Web Hosting & Bandwidth Costs,26/05/2016 3:03:00 PM,Components for information technology or broad...,Eye Media Australia Pty Ltd,NaN,EYE MEDIA AUSTRALIA PTY LTD,14
1,CN2707381,44012,Extension of the iVEC UQ RDSI Existing Disk Ar...,12/08/2015 12:00:00 AM,Components for information technology or broad...,DataDirect Networks,Recovered from Discards; Supplier provides sto...,DDN AUSTRALIA PTY LIMITED,9
2,CN2745091,9793,IT Professional Services,5/01/2015 10:01:00 AM,Information technology consultation services,Redcore Pty Ltd,NaN,REDCORE PTY LTD,11
3,CN2748341,6061,ICT Contractors,14/01/2015 11:28:00 AM,Computer services,Southern Cross Computing Pty Ltd,NaN,SOUTHERN CROSS COMPUTING PTY LIMITED,13
4,CN2748891,21412,ICT Contractors,5/01/2015 11:21:00 AM,Computer services,Horizon Consulting Pty Ltd,NaN,HORIZON CONSULTING PTY LTD,13


In [4]:
#### discards in the last 3 years.
discards = pd.read_csv('3ydiscards.csv')
(discards.head())

,Cnid,Publish Date,Description,Category,SON ID,Supplier Name,Comment
0,CN1094702,26/05/2016 3:10:00 PM,Provision of Financial Consultancy Services,Accounting services,SON241274,Synergy Group Australia Ltd,NaN
1,CN1907701,1/02/2016 11:07:00 AM,Internal Audit,Audit services,NaN,Moore Stephens,NaN
2,CN2032461,26/05/2016 2:25:00 PM,Registar Liaison Officer,Temporary personnel services,NaN,Attorney General's Department CPM,NaN
3,CN2439052,12/02/2015 10:19:00 AM,Printing and Distribution 2014,Printing,NaN,CanPrint Communications Pty Ltd,NaN
4,CN2542831,25/03/2015 11:38:00 AM,Management Services,Management support services,SON1180562,DFP Recruitment Services Pty Ltd,NaN


# Stage 2: Modeling fitting description based model.
We assume that Master contract match and ICT supplier categorisation is reliable.
We start from modelling descriptions.
#### Extract useful columns

In [5]:
# contract
contract_des = contract[['Cnid','Title','IdCat']]

In [6]:
contract_des.columns=['Cnid','Description','idCat']
contract_des.head()

,Cnid,Description,idCat
0,CN1148392,Provision of Web Hosting & Bandwidth Costs,14
1,CN2707381,Extension of the iVEC UQ RDSI Existing Disk Ar...,9
2,CN2745091,IT Professional Services,11
3,CN2748341,ICT Contractors,13
4,CN2748891,ICT Contractors,13


In [7]:
contract_des[contract_des.Description.isnull()]

,Cnid,Description,idCat
10867,CN3191352,NaN,40


In [8]:
contract_des.fillna(value="NODesc",inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
len(contract_des) # 55702 raw contract in total.

55702

In [26]:
discards_des =discards[['Cnid','Description']]
discards_des[discards_des.Description.isnull()]

,Cnid,Description
70659,CN3328465,NaN
91960,CN3360215,NaN


In [10]:
discards_des.fillna(value="NODesc",inplace=True) # remember to fillna otherwise it would cause errors in the code below.
discards_des.head()
# categories can fillna after wards when joined with contracts when needed.

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Cnid,Description
0,CN1094702,Provision of Financial Consultancy Services
1,CN1907701,Internal Audit
2,CN2032461,Registar Liaison Officer
3,CN2439052,Printing and Distribution 2014
4,CN2542831,Management Services


Give discards category of 99

In [11]:
idDis = pd.Series(np.ones(len(discards_des)))
discards_des = pd.concat([discards_des,idDis],axis=1,ignore_index=True)

In [12]:
discards_des.columns= ['Cnid','Description','idCat']
discards_des.idCat = 99
discards_des.head()

,Cnid,Description,idCat
0,CN1094702,Provision of Financial Consultancy Services,99
1,CN1907701,Internal Audit,99
2,CN2032461,Registar Liaison Officer,99
3,CN2439052,Printing and Distribution 2014,99
4,CN2542831,Management Services,99


### make train test set for each.
Separate df into 20% and rest. So is discards

In [13]:
# need to import sub packages explicitly
from sklearn import model_selection

In [14]:
contract_train,contract_test = sklearn.model_selection.train_test_split(contract_des,test_size=0.2, random_state=1)

In [15]:
discards_train,discards_test = sklearn.model_selection.train_test_split(discards_des,test_size=0.2, random_state=1)

* When description only method, join two dataframe into one for both contract and discards. 

In [16]:
df_train = pd.concat([contract_train,discards_train],join='outer',ignore_index=True)
df_train.head()

,Cnid,Description,idCat
0,CN3339766,Human Systems Integration Course,19
1,CN3419927,Software Upgrade,17
2,CN3378922,ICT Contractor Services,13
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,13
4,CN3295946,ICT Services,17


In [17]:
df_test = pd.concat([contract_test,discards_test],join='outer',ignore_index=True)
df_test.head()

,Cnid,Description,idCat
0,CN3348178,Broadcast Video Editing and Archiving System,43
1,CN3110212,Distributed Computing Central Services,14
2,CN3442793,Automated Test Engineer,13
3,CN3422590,Software Services,28
4,CN3428960,VC System,43


Insert one more column,if category is not 99 or 52, IT.

In [18]:
df_train.insert(loc=3,column='yesIT',value= 1)

In [19]:
df_train.head()

,Cnid,Description,idCat,yesIT
0,CN3339766,Human Systems Integration Course,19,1
1,CN3419927,Software Upgrade,17,1
2,CN3378922,ICT Contractor Services,13,1
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,13,1
4,CN3295946,ICT Services,17,1


In [20]:
df_train.yesIT[df_train.idCat.isin([52,99])]=0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
print('NON-IT',
    df_train[df_train.yesIT==0].head(),'\n','\n',
     len(df_train[df_train.yesIT==0]),'\n')
print('IT',df_train[df_train.yesIT==1].head(),'\n','\n',
 len(df_train[df_train.yesIT==1]))

NON-IT           Cnid                             Description  idCat  yesIT
289  CN2964562  Project Manager - Questacon Facilities     52      0
409  CN3203062              Acute Care Mapping Project     52      0
466  CN3199632                     Forensic Accounting     52      0
515  CN3283162   Project Officer - Business Continuity     52      0
648  CN3281406                    Labour Hire Services     52      0 
 
 124971 

IT         Cnid                                        Description  idCat  yesIT
0  CN3339766                   Human Systems Integration Course     19      1
1  CN3419927                                   Software Upgrade     17      1
2  CN3378922                            ICT Contractor Services     13      1
3  CN3432035  DVA ICTSB 2016/157 - System Analyst - ICT Plan...     13      1
4  CN3295946                                       ICT Services     17      1 
 
 43886


### Step2: Normalisation of words - for training set only.

__ We need to standardise words with the following features: __

* retain upper case features for certain words with different meanings when lowercased like ['IT',] - replace with word: infotech as there is no other matches.

Tokenize each description before lemmatizing them.

In [27]:
df_train.head(10)

,Cnid,Description,idCat,yesIT
0,CN3339766,Human Systems Integration Course,19,1
1,CN3419927,Software Upgrade,17,1
2,CN3378922,ICT Contractor Services,13,1
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,13,1
4,CN3295946,ICT Services,17,1
5,CN3396801,Information retrieval systems,29,1
6,CN2979762,Printers & Licences,8,1
7,CN3421256,Laptops - high end,4,1
8,CN3330933,Provision of Computer Programmers,13,1
9,CN3318967,Provision of infotech Contractor Services,13,1


In [23]:
# replace 'IT' with 'infotech'
IT_regex = r'(?<=(\b))IT(?=(\b))'
df_train.Description = df_train.Description.str.replace(IT_regex, 'infotech',case=True)

In [24]:
# make sure no IT is missed.
print(df_train[df_train.Description.str.contains('infotech')].count())
print(df_train[df_train.Description.str.contains('IT')].count())
print(df_train[df_train.Description.str.contains(' IT ')].head())

Cnid           3465
Description    3465
idCat          3465
yesIT          3465
dtype: int64
Cnid           923
Description    923
idCat          923
yesIT          923
dtype: int64
Empty DataFrame
Columns: [Cnid, Description, idCat, yesIT]
Index: []


* We extract description column to process before joining back to the dataframe.

Lemmatization considers upper case as special terminologies, so still lower cased every thing.

In [ ]:
# tokenize sentences.
title_tokenised = df_train.Description.str.lower().apply(nltk.word_tokenize)
title_tokenised.head()

In [ ]:
# Create function to Lemmatize each entry but do not return to sentence.
def lem_cont(tokens):
    wnl = nltk.WordNetLemmatizer()
    lemmed = [wnl.lemmatize(t) for t in tokens]
    return lemmed

In [ ]:
title_token_lem = title_tokenised.apply(lem_cont)
title_token_lem.head()

In [ ]:
#print([nltk.WordNetLemmatizer().lemmatize(x) for x in ['Provision','Services','services','infotech','info']])

In [ ]:
# save to disk lemtoken as xlsx - so next time can start from here.
title_lemtoken.to_excel(excel_writer='title_lemtoken.xlsx',header='lem_title')
title_lemtoken.to_csv('title_lemtoken.csv',header='lem_title')

Try Stemming
> following code is raw text so that they are not executed

## Step3: Removal of stop words

> start by loading the 'title_lemtoken.csv' that contains the tokenised entries.

In [ ]:
from nltk.corpus import stopwords

In [ ]:
filtered = [w for w in title_lemtoken if not w in stopwords.words('english')]

## Change categories results matrix.

## Step4: Descriptive data

Some high-level descriptive data that measures popular words, etc.

In [ ]:
tokensstr = ' '.join(?)

In [ ]:
tokens=nltk.word_tokenize(tokensstr)

In [ ]:
tokens[:30]

In [ ]:
from collections import Counter

In [ ]:
counter = Counter(tokens)
counter.most_common(20)

## Step5: Vectorising titles

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

## Step6: similarity score evaluation